In [16]:
import sys
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn import preprocessing
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [17]:
df = pd.read_csv('ai_mid3.csv', delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())

(506, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   INDUS    506 non-null    float64
 2   RM       506 non-null    float64
 3   TAX      506 non-null    float64
 4   PTRATIO  506 non-null    float64
 5   LSTAT    506 non-null    float64
 6   MEDV     506 non-null    float64
dtypes: float64(7)
memory usage: 27.8 KB
None


,CRIM,INDUS,RM,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,2.31,6.575,296.0,15.3,4.98,24.0
1,0.02731,7.07,6.421,242.0,17.8,9.14,21.6
2,0.02729,7.07,7.185,242.0,17.8,4.03,34.7
3,0.03237,2.18,6.998,222.0,18.7,2.94,33.4
4,0.06905,2.18,7.147,222.0,18.7,5.33,36.2


In [18]:
df_corr = df.corr(); display(df_corr)

,CRIM,INDUS,RM,TAX,PTRATIO,LSTAT,MEDV
CRIM,1.000000,0.406583,-0.219247,0.582764,0.289946,0.455621,-0.388305
INDUS,0.406583,1.000000,-0.391676,0.720760,0.383248,0.603800,-0.483725
RM,-0.219247,-0.391676,1.000000,-0.292048,-0.355501,-0.613808,0.695360
TAX,0.582764,0.720760,-0.292048,1.000000,0.460853,0.543993,-0.468536
PTRATIO,0.289946,0.383248,-0.355501,0.460853,1.000000,0.374044,-0.507787
LSTAT,0.455621,0.603800,-0.613808,0.543993,0.374044,1.000000,-0.737663
MEDV,-0.388305,-0.483725,0.695360,-0.468536,-0.507787,-0.737663,1.000000


In [19]:
ser_medv_corr = df.corrwith(df['MEDV']); print(ser_medv_corr)

CRIM      -0.388305
INDUS     -0.483725
RM         0.695360
TAX       -0.468536
PTRATIO   -0.507787
LSTAT     -0.737663
MEDV       1.000000
dtype: float64


In [20]:
X_all_org = df.loc[:, 'TAX':'LSTAT']  # explanatory variables
y = df['MEDV']  # objective variable
print('X_all_org:', X_all_org.shape)
display(X_all_org.head())
print('y:', y.shape)
print(y.head())

X_all_org: (506, 3)


,TAX,PTRATIO,LSTAT
0,296.0,15.3,4.98
1,242.0,17.8,9.14
2,242.0,17.8,4.03
3,222.0,18.7,2.94
4,222.0,18.7,5.33


y: (506,)
0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64


In [21]:
X_all = pd.get_dummies(X_all_org, drop_first=True)
print('X_all:', X_all.shape)
display(X_all.head())

X_all: (506, 3)


,TAX,PTRATIO,LSTAT
0,296.0,15.3,4.98
1,242.0,17.8,9.14
2,242.0,17.8,4.03
3,222.0,18.7,2.94
4,222.0,18.7,5.33


In [22]:
X_scaled = preprocessing.scale(X_all)
y_scaled = preprocessing.scale(y)
model = sm.OLS(y_scaled, X_scaled)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.607
Model:                            OLS   Adj. R-squared (uncentered):              0.604
Method:                 Least Squares   F-statistic:                              258.6
Date:                Fri, 04 Jun 2021   Prob (F-statistic):                   1.69e-101
Time:                        09:58:50   Log-Likelihood:                         -481.91
No. Observations:                 506   AIC:                                      969.8
Df Residuals:                     503   BIC:                                      982.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [23]:
dfX_scaled = pd.DataFrame(X_scaled, columns=X_all.columns)
dfy_scaled = pd.Series(y_scaled, name=y.name)

In [24]:
exogs = ['TAX', 'PTRATIO', 'LSTAT']
X_final_scaled = dfX_scaled[exogs]
model_final_scaled = sm.OLS(y_scaled, X_final_scaled)
results_final_scaled = model_final_scaled.fit()
print(results_final_scaled.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.607
Model:                            OLS   Adj. R-squared (uncentered):              0.604
Method:                 Least Squares   F-statistic:                              258.6
Date:                Fri, 04 Jun 2021   Prob (F-statistic):                   1.69e-101
Time:                        09:58:50   Log-Likelihood:                         -481.91
No. Observations:                 506   AIC:                                      969.8
Df Residuals:                     503   BIC:                                      982.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [26]:
print(results_final_scaled.params)

TAX        0.003401
PTRATIO   -0.270603
LSTAT     -0.638295
dtype: float64
